In [2]:
import tensorflow as tf

# Appréhender le fonctionnement de TF

In [9]:
# Créer des Constantes (Noeuds n'acceptant pas d'entrée)
node1 = tf.constant(3.0, dtype=tf.float32) # tf.constant initialise le noeud
node2 = tf.constant(4.0)                   # implicitement un tf.float32
print(node1, node2)

Tensor("Const_12:0", shape=(), dtype=float32) Tensor("Const_13:0", shape=(), dtype=float32)


In [10]:
# Lancer une Session (computationnal graph), encapsulant le contrôle et l'état des tenseurs
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [11]:
# Réaliser des Opérations (qui sont aussi des Noeuds)
from __future__ import print_function
node3 = tf.add(node1,node2)
print("node3:", node3)
print("sess.run(node3)", sess.run(node3))

node3: Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3) 7.0


In [13]:
# Créer des Placeholders (variables auxquelles on pourra attribuer des valeurs)
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b # + est un raccourci pour tf.add(a,b)
add_and_triple = adder_node * 3

print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))
print(sess.run(add_and_triple, {a: 3, b: 4.5}))

7.5
[ 3.  7.]
22.5


In [14]:
# Créer des Variables (dont les sorties provoquées par une même entrée peuvent être différentes)
W = tf.Variable([.3], dtype=tf.float32)   # 0.3 correponds à sa valeur initiale
b = tf.Variable([-0.3], dtype=tf.float32) # tf.Variable n'initialise pas le noeud
x = tf.placeholder(tf.float32)
linear_model = W*x + b

# Pour initialiser des Variables :
init = tf.global_variables_initializer()
sess.run(init)

In [15]:
# Evaluer linear_model pour plusieurs valeurs du Placeholder x
print(sess.run(linear_model, {x: [1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [17]:
# Compléter le modèle linéaire
y = tf.placeholder(tf.float32)               # Contiendra les valeurs désirées

# Calculer le coût :
squared_deltas = tf.square(linear_model - y) # (linear - y) renvoit un vecteur contenant toutes les différences, tf.square met au carré chaucune de ces valeurs
loss = tf.reduce_sum(squared_deltas)         # tf.reduce_sum somme toutes les valeurs de Arg1

print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

23.66


In [18]:
# Optimiser manuellement le modèle pour réduire le coût à 0
fixW = tf.assign(W, [-1]) #tf.assign assigne à Arg1 une nouvelle valeur Arg2
fiwb = tf.assign(b, [1])
sess.run([fixW, fiwb])

print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

0.0


In [19]:
# Optimiser automatiquement le modèle en utilisant un Optimiseur (ici gradient descent, le plus simple)
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

sess.run(init)        # réinitialise les valeurs à celles par défaut, incorrectes
for i in range(1000): # Arg1 : nombre d'itérations à l'apprentissage
    sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})
    
print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


## tf.estimator
Library TF haut-niveau simplifiant les mécanismes de ML (réaliser des entrainements et des évaluations, gérer les jeux de données)

In [20]:
import numpy as np
import tensorflow as tf

In [22]:
# Définir la liste de features
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# Un estimateur permet d'invoquer l'entrainement (fitting) et l'évaluation (inférence)
# Nombreux types d'estimateurs, ici on utilise un régresseur linéaire
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# Définition des jeux de données (différent pour l'entrainement et l'évaluation)
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])
input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_train}, 
                                               y_train, 
                                               batch_size=4,
                                               num_epochs=None,
                                               shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_train}, 
                                                     y_train, 
                                                     batch_size=4,
                                                     num_epochs=1000,
                                                     shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_eval}, 
                                                    y_eval, 
                                                    batch_size=4,
                                                    num_epochs=1000,
                                                    shuffle=False)

# Invocation de l'entrainement (en Arg2 étapes)
estimator.train(input_fn=input_fn, steps=1000)

# Evaluation de l'efficacité du modèle
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics : %r"% train_metrics)
print("eval metrics : %r"% eval_metrics)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_session_config': None, '_tf_random_seed': None, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_model_dir': '/tmp/tmponues3nq', '_is_chief': True, '_num_ps_replicas': 0, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_num_worker_replicas': 1, '_save_checkpoints_secs': 600, '_master': '', '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1956b9d4e0>, '_service': None}


INFO:tensorflow:Using config: {'_session_config': None, '_tf_random_seed': None, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_model_dir': '/tmp/tmponues3nq', '_is_chief': True, '_num_ps_replicas': 0, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_num_worker_replicas': 1, '_save_checkpoints_secs': 600, '_master': '', '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1956b9d4e0>, '_service': None}


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:Finished evaluation at 2018-01-17-13:33:12


INFO:tensorflow:Finished evaluation at 2018-01-17-13:33:12


INFO:tensorflow:Saving dict for global step 1000: average_loss = 0.00252721, global_step = 1000, loss = 0.0101089


INFO:tensorflow:Saving dict for global step 1000: average_loss = 0.00252721, global_step = 1000, loss = 0.0101089


train metrics : {'average_loss': 6.6248995e-10, 'global_step': 1000, 'loss': 2.6499598e-09}
eval metrics : {'average_loss': 0.0025272148, 'global_step': 1000, 'loss': 0.010108859}


## tf.estimator (modèle personnalisé)
tf.estimator contient un certain nombre de modèles prédéfinis, mais il est aussi possible créer nos propres modèles personnalisés.  
Pour cela, il faut fournir à tf.estimator.Estimator une fonction modele_fn qui décrit à tf.estimator comment évaluer les prédictions, les étapes d'entrainement et la fonction de perte. 

In [23]:
import numpy as np
import tensorflow as tf

In [27]:
# Définir la liste de features
def model_fn(features, labels, mode):
    # Construire un modèle linéaire et prédire les valeurs
    W = tf.get_variable("W", [1], dtype=tf.float64)
    b = tf.get_variable("b", [1], dtype=tf.float64)
    y = W*features['x'] + b
    
    # Calcul perte
    loss = tf.reduce_sum(tf.square(y - labels))
    
    # Entrainement
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(optimizer.minimize(loss), tf.assign_add(global_step, 1))
    
    #EstimatorSpec rassemble les sous-parties pour les rendre fonctionnelles
    return tf.estimator.EstimatorSpec(mode=mode,
                                     predictions=y,
                                     loss=loss,
                                     train_op=train)

estimator = tf.estimator.Estimator(model_fn=model_fn)

# Définition des jeux de données
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])
input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_train},
                                             y_train,
                                             batch_size=4,
                                             num_epochs=None,
                                             shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_train},
                                             y_train,
                                             batch_size=4,
                                             num_epochs=1000,
                                             shuffle=False)

eval_input_fn = tf.estimator.inputs.numpy_input_fn({"x": x_eval},
                                             y_eval,
                                             batch_size=4,
                                             num_epochs=1000,
                                             shuffle=False)

# Entrainement
estimator.train(input_fn=input_fn, steps=1000)

# Evaluation de l'efficacité du modèle
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics : %r"% train_metrics)
print("eval metrics : %r"% eval_metrics)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_session_config': None, '_tf_random_seed': None, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_model_dir': '/tmp/tmps9jjsjn0', '_is_chief': True, '_num_ps_replicas': 0, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_num_worker_replicas': 1, '_save_checkpoints_secs': 600, '_master': '', '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1955998d68>, '_service': None}


INFO:tensorflow:Using config: {'_session_config': None, '_tf_random_seed': None, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_model_dir': '/tmp/tmps9jjsjn0', '_is_chief': True, '_num_ps_replicas': 0, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_num_worker_replicas': 1, '_save_checkpoints_secs': 600, '_master': '', '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1955998d68>, '_service': None}


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:Finished evaluation at 2018-01-17-14:19:28


INFO:tensorflow:Finished evaluation at 2018-01-17-14:19:28


INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 0.0101014


INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 0.0101014


train metrics : {'global_step': 1000, 'loss': 1.289916e-10}
eval metrics : {'global_step': 1000, 'loss': 0.01010139}


# Utiliser TF sur MNIST

In [29]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MINST_data/", one_hot=True)

Extracting MINST_data/train-images-idx3-ubyte.gz
Extracting MINST_data/train-labels-idx1-ubyte.gz
Extracting MINST_data/t10k-images-idx3-ubyte.gz
Extracting MINST_data/t10k-labels-idx1-ubyte.gz


In [31]:
# Initialisation des paramètres
x = tf.placeholder(tf.float32, [None,784]) # 784 corresponds à la taille de l'array d'une image MNIST aplatie
W = tf.Variable(tf.zeros([784, 10]))       # 10 corresponds au nombre de classes
b = tf.Variable(tf.zeros([10]))

# Définition du modèle (en une seule ligne)
y = tf.nn.softmax(tf.matmul(x, W) + b)     # matmul : matrice multiplication

# Calcul de la cross-entropy
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

# Entrainement
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [32]:
# Lancer le modèle
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [34]:
# Evaluer l'efficacité du modèle
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})) # %precision (state-of-the-art models : 0.997)

0.9149


# DeepLearning pour MNIST via TF

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [10]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10]) # labels

W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
sess.run(tf.global_variables_initializer())

In [11]:
y = tf.matmul(x,W) + b

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

In [12]:
# Entrainement
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

for _ in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [13]:
# Evaluer l'efficacité du modèle
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9207


### Construire un réseau convolutif multicouche
Réseau composée de [neurones ReLu](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)

In [4]:
# Fonction permettant d'initialisation des poids
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

# Fonction permettant d'insérer un biais/bruits aux poids
# Le bruit évite d'obtenir des poids symétiques et égaux à 0
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [7]:
# Fonction permettant d'appeler la convolution
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# Fonction permettant d'appeler le max pooling
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides = [1, 2, 2, 1], padding='SAME')

In [14]:
# Première couche convolutive
# Calcule 32 caractèristiques pour chacun des patchs 5x5
# Obtient un tenseur de taille [taille patch, taille patch, nb inputs, nb outputs], ici : [5, 5, 1, 32]

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# Reshape x pour être un tenseur 4d
x_image = tf.reshape(x, [-1, 28, 28, 1])

# Réaliser la convolution via des neurones ReLu
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1) # réduit l'image à une taille 14x14

In [16]:
# Seconde couche convolutive
# Calcule 64 caractèristiques pour chacun des paths 5x5

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_poo1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2) # réduit l'image à une taille 7x7

In [17]:
# Couche fully-connected de 1024 neurones

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [18]:
# Application d'un dropout (source en satellite de DAUCE-W7.ipynb) avant la couche de sortie
# Méthode pour diminuer le risque de sur-apprentissage

# keep_prob conserve la probabilité que la sortie d'un neurone soit conservé durant le dropout (permet d'active le dropout uniquement pendant l'apprentissage, et pas pendant le test)
keep_prob = tf.placeholder(tf.float32)
# tf.nn.dropout automatically handles scaling neuron outputs in addition to maskin them (allow dropout to work without any additional scaling)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [19]:
# Couche de sortie

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [20]:
# Entrainer et évaluer le modèle
# Utilisation d'un optimisateur ADAM (plus sophistiqué que l'optimisateur en descente de gradient)
# Le bloc "with tf.Session" permet de détruire la session une fois celle-ci achevée (optimisation des ressources)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
            print('step %d, training accuracy %g' % (i,train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
        
    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.22
step 100, training accuracy 0.9
step 200, training accuracy 0.94
step 300, training accuracy 0.96
step 400, training accuracy 0.9
step 500, training accuracy 0.94
step 600, training accuracy 0.92
step 700, training accuracy 0.98
step 800, training accuracy 0.96
step 900, training accuracy 0.98
step 1000, training accuracy 0.96
step 1100, training accuracy 0.96
step 1200, training accuracy 0.96
step 1300, training accuracy 1
step 1400, training accuracy 1
step 1500, training accuracy 0.96
step 1600, training accuracy 0.98
step 1700, training accuracy 0.98
step 1800, training accuracy 1
step 1900, training accuracy 1
test accuracy 0.9753


# Visualiser l'activité de TF via TensorBoard

In [4]:
"""A simple MNIST classifier which displays summaries in TensorBoard.
This is an unimpressive MNIST model, but it is a good example of using
tf.name_scope to make a graph legible in the TensorBoard graph explorer, and of
naming summary tags so that they are grouped meaningfully in TensorBoard.
It demonstrates the functionality of every TensorBoard dashboard.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
import sys

import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

FLAGS = None


def train():
  # Import data
  mnist = input_data.read_data_sets(FLAGS.data_dir,
                                    one_hot=True,
                                    fake_data=FLAGS.fake_data)

  sess = tf.InteractiveSession()
  # Create a multilayer model.

  # Input placeholders
  with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')

  with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_shaped_input, 10)

  # We can't initialize these variables to 0 - the network will get stuck.
  def weight_variable(shape):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

  def bias_variable(shape):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

  def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
      mean = tf.reduce_mean(var)
      tf.summary.scalar('mean', mean)
      with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
      tf.summary.scalar('stddev', stddev)
      tf.summary.scalar('max', tf.reduce_max(var))
      tf.summary.scalar('min', tf.reduce_min(var))
      tf.summary.histogram('histogram', var)

  def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    """Reusable code for making a simple neural net layer.
    It does a matrix multiply, bias add, and then uses ReLU to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read,
    and adds a number of summary ops.
    """
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
      # This Variable will hold the state of the weights for the layer
      with tf.name_scope('weights'):
        weights = weight_variable([input_dim, output_dim])
        variable_summaries(weights)
      with tf.name_scope('biases'):
        biases = bias_variable([output_dim])
        variable_summaries(biases)
      with tf.name_scope('Wx_plus_b'):
        preactivate = tf.matmul(input_tensor, weights) + biases
        tf.summary.histogram('pre_activations', preactivate)
      activations = act(preactivate, name='activation')
      tf.summary.histogram('activations', activations)
      return activations

  hidden1 = nn_layer(x, 784, 500, 'layer1')

  with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.summary.scalar('dropout_keep_probability', keep_prob)
    dropped = tf.nn.dropout(hidden1, keep_prob)

  # Do not apply softmax activation yet, see below.
  y = nn_layer(dropped, 500, 10, 'layer2', act=tf.identity)

  with tf.name_scope('cross_entropy'):
    # The raw formulation of cross-entropy,
    #
    # tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.softmax(y)),
    #                               reduction_indices=[1]))
    #
    # can be numerically unstable.
    #
    # So here we use tf.nn.softmax_cross_entropy_with_logits on the
    # raw outputs of the nn_layer above, and then average across
    # the batch.
    diff = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)
    with tf.name_scope('total'):
      cross_entropy = tf.reduce_mean(diff)
  tf.summary.scalar('cross_entropy', cross_entropy)

  with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(FLAGS.learning_rate).minimize(
        cross_entropy)

  with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
      correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    with tf.name_scope('accuracy'):
      accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  tf.summary.scalar('accuracy', accuracy)

  # Merge all the summaries and write them out to
  # /tmp/tensorflow/mnist/logs/mnist_with_summaries (by default)
  merged = tf.summary.merge_all()
  train_writer = tf.summary.FileWriter(FLAGS.log_dir + '/train', sess.graph)
  test_writer = tf.summary.FileWriter(FLAGS.log_dir + '/test')
  tf.global_variables_initializer().run()

  # Train the model, and also write summaries.
  # Every 10th step, measure test-set accuracy, and write test summaries
  # All other steps, run train_step on training data, & add training summaries

  def feed_dict(train):
    """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
    if train or FLAGS.fake_data:
      xs, ys = mnist.train.next_batch(100, fake_data=FLAGS.fake_data)
      k = FLAGS.dropout
    else:
      xs, ys = mnist.test.images, mnist.test.labels
      k = 1.0
    return {x: xs, y_: ys, keep_prob: k}

  for i in range(FLAGS.max_steps):
    if i % 10 == 0:  # Record summaries and test-set accuracy
      summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
      test_writer.add_summary(summary, i)
      print('Accuracy at step %s: %s' % (i, acc))
    else:  # Record train set summaries, and train
      if i % 100 == 99:  # Record execution stats
        run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
        run_metadata = tf.RunMetadata()
        summary, _ = sess.run([merged, train_step],
                              feed_dict=feed_dict(True),
                              options=run_options,
                              run_metadata=run_metadata)
        train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
        train_writer.add_summary(summary, i)
        print('Adding run metadata for', i)
      else:  # Record a summary
        summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
        train_writer.add_summary(summary, i)
  train_writer.close()
  test_writer.close()


def main(_):
  if tf.gfile.Exists(FLAGS.log_dir):
    tf.gfile.DeleteRecursively(FLAGS.log_dir)
  tf.gfile.MakeDirs(FLAGS.log_dir)
  train()


if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  parser.add_argument('--fake_data', nargs='?', const=True, type=bool,
                      default=False,
                      help='If true, uses fake data for unit testing.')
  parser.add_argument('--max_steps', type=int, default=1000,
                      help='Number of steps to run trainer.')
  parser.add_argument('--learning_rate', type=float, default=0.001,
                      help='Initial learning rate')
  parser.add_argument('--dropout', type=float, default=0.9,
                      help='Keep probability for training dropout.')
  parser.add_argument(
      '--data_dir',
      type=str,
      default=os.path.join(os.getenv('TEST_TMPDIR', '/tmp'),
                           'tensorflow/mnist/input_data'),
      help='Directory for storing input data')
  parser.add_argument(
      '--log_dir',
      type=str,
      default=os.path.join(os.getenv('TEST_TMPDIR', '/tmp'),
                           'tensorflow/mnist/logs/mnist_with_summaries'),
      help='Summaries log directory')
  FLAGS, unparsed = parser.parse_known_args()
tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz
Accuracy at step 0: 0.104
Accuracy at step 10: 0.6871
Accuracy at step 20: 0.8262
Accuracy at step 30: 0.8663
Accuracy at step 40: 0.8735
Accuracy at step 50: 0.8879
Accuracy at step 60: 0.903
Accuracy at step 70: 0.9027
Accuracy at step 80: 0.9079
Accuracy at step 90: 0.9137
Adding run metadata for 99
Accuracy at step 100: 0.9118
Accuracy at step 110: 0.9114
Accuracy at step 120: 0.9181
Accuracy at step 130: 0.9218
Accuracy at step 140: 0.9258
Accuracy at step 150: 0.9251
Accuracy at step 160: 0.9182
Accuracy at step 170: 0.9322
Accuracy at step 180: 0.9329
Accuracy at step 190: 0.9313
Adding run metadata for 199
Accuracy at step 200: 0.935
Accuracy at step 210: 0.9336
Accuracy at step 220: 0.9389
Accuracy a

SystemExit: 

/home/pimt/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
# Reinitialiser les valeurs de TensorBoard (permet de relancer la cellule précédente)
tf.reset_default_graph()

NameError: name 'python' is not defined